<a href="https://colab.research.google.com/github/giyuhwang/calculatorapp/blob/master/UTRA_ART_CV_Chessman_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#UTRA ART CV Onboarding - Mini Project Task
# Chessman Type Detector

Author: Gil Hwang

Task: Chess Type Detector

Start Date: Oct 22 2022

Dataset Reference: https://www.kaggle.com/datasets/niteshfre/chessman-image-dataset

Tutorial Reference: https://www.youtube.com/watch?v=c36lUUr864M

In [ ]:
# Import Necessary Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Mount to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Class Definition for neural network
class ChessNet(nn.Module):
  def __init__(self):
    """ Constructor of ChessNet
    """
    super(ChessNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 6)

  def forward(self, x):
    """ Forward definition of ChessNet
    """
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16 * 5 * 5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
def train_net(model, batch_size, learning_rate, num_epochs):
  ########################################################
  # Get data loaders

  # Set dataset tranformation
  transform = transforms.Compose([transforms.ToTensor(),
                                  transforms.Resize((32, 32)),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  # Folder Path to dataset
  train_path = '/content/drive/MyDrive/Colab Notebooks/UTRA ART - CV/chess_dataset/Chess'
  test_path = '/content/drive/MyDrive/Colab Notebooks/UTRA ART - CV/chess_test_data'

  # Retrieve train dataset
  train_dataset = torchvision.datasets.ImageFolder(root = train_path,
                                                  transform = transform)
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle = True)

  # Retrieve test dataset
  test_dataset = torchvision.datasets.ImageFolder(root = test_path,
                                                  transform = transform)
  test_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle = True)

  #########################################################
  # Local Variables
  classes = ('bishop', 'king', 'knight', 'pawn', 'queen', 'rook')

  #########################################################
  # Define loss function and optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

  #########################################################
  # Train network
  print('Start Training.')

  n_total_steps = len(train_loader)
  for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
      # Link images and labels to be calculated in GPU if available
      images = images.to(device)
      labels = labels.to(device)

      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)

      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Print report every 2000 iterations
      if (i+1) % 2000 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {.4f}'.format(epoch+1, 
                                                            num_epochs,
                                                            i+1,
                                                            n_total_steps,
                                                            loss.item()))
    
  print('Finished Training.')

  with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(6)]
    n_class_samples = [0 for i in range(6)]

    # Get accuracy
    for images, labels in test_loader:
      # Do forward pass
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)

      # Get predictions for overall class
      _, predicted = torch.max(outputs, 1)
      n_samples += labels.size(0)
      n_correct += (predicted == labels).sum().item()

      # Get prediction for each class
      for i in range(batch_size):
        label = labels[i]
        pred = predicted[i]
        if (label == pred):
          n_class_correct[label] += 1
        n_class_samples[label] += 1

    
    # Compute accuracy

      





In [ ]:
# Set hyper parameters
num_epochs = 5
batch_size = 16
learning_rate = 0.001

# Create model instance
my_net = ChessNet().to(device)

# Train network
train_net(my_net, batch_size, learning_rate, num_epochs)

Start Training.


KeyboardInterrupt: ignored